## For loop 속도 개선하기

강의자료 출처
- https://blog.fearcat.in/a?ID=00900-6997c6fb-2680-4531-af1d-73eeccce74ef
- https://aldente0630.github.io/data-science/2018/08/05/a-beginners-guide-to-optimizing-pandas-code-for-speed.html

In [2]:
import pandas as pd
import numpy as np
from math import *

### Read in the data

익스피디아 개발자 사이트에서 제공한 뉴욕 주 내 모든 호텔 좌표가 들어있는 데이터셋

NameError: name 'os' is not defined

In [4]:
import pandas as pd
df = pd.read_csv('C://Users//jiui//Desktop//mining//dataset//new_york_hotels.csv', encoding='cp1252')

In [5]:
df.head()

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900


## Haversine definition

두 위치 사이의 거리를 계산하는 함수
- https://stricky.tistory.com/284

In [6]:
#출발지와 도착지의 위경도 

def haversine(lat1, lon1, lat2, lon2):
    miles_constant = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    mi = miles_constant * c
    return mi

## Task 

어떤 위치, (40.671, -73.985)에서 `df`에 존재하는 모든 호텔까지의 거리를 구해봅시다

## Looping Haversine

In [7]:
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        distance_list.append(d)
    return distance_list

In [8]:
%%timeit

# Haversine 반복 함수 실행하기
df['distance'] = haversine_looping(df)

134 ms ± 752 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Iterrows Haversine

반복문을 돌려야 할 때 iterrows() 메서드를 사용하는 건 행을 반복하기 위한 더 좋은 방법이다. iterrows()는 데이터 프레임의 행을 반복하며 행 자체를 포함하는 객체에 덧붙여 각 행의 색인을 반환하는 제너레이터다. iterrows()는 판다스 데이터 프레임과 함께 작동하게끔 최적화되어 있으며 표준 함수 대부분을 실행하는 데 가장 효율적인 방법은 아니지만(나중에 자세히 설명) 단순 반복보다는 상당히 개선되었다. 예제의 경우 iterrows()는 행을 수동으로 반복하는 것보다 거의 똑같은 문제를 약 4배 빠르게 해결한다.

In [9]:


%%timeit
# Haversine applied on rows via iteration
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985, row['latitude'], row['longitude']))
df['distance'] = haversine_series

33 ms ± 205 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
import pandas as pd
df2 = pd.DataFrame({'A':[1,2,3],'B':['a','b','c']})
print(df2)

   A  B
0  1  a
1  2  b
2  3  c


## Apply Haversine on rows

`iterrows()`보다 더 좋은 옵션은 데이터 프레임의 특정 축(행 또는 열을 의미)을 따라 함수를 적용하는 `apply()` 메서드를 사용하는 것이다. `apply()`는 본질적으로 행을 반복하지만 Cython에서 이터레이터를 사용하는 것 같이 내부 최적화를 다양하게 활용하므로 `iterrows()`보다 훨씬 효율적이다.

익명의 람다 함수를 사용하여 Haversine 함수를 각 행에 적용하며 각 행의 특정 셀을 함수 입력값으로 지정할 수 있다. 람다 함수는 판다스가 행(축 = 1)과 열(축 = 0) 중 어디에 함수를 적용할지 정할 수 있게 축 매개 변수를 마지막에 포함한다.

### Timing "apply"

In [11]:
%timeit df['distance'] =\
df.apply(lambda row: haversine(40.671, -73.985,\
                               row['latitude'], row['longitude']), axis=1)

17.8 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Vectorized implementation of Haversine applied on Pandas series

#### Timing vectorized implementation

함수 수행의 반복량 줄이는 방법을 이해하기 위해 판다스의 기본 단위, 데이터 프레임과 시리즈가 모두 배열 기반임을 알아두자. 기본 단위의 내부 구조는 개별 값(스칼라라고 함)마다 순차적으로 작동하는 대신 전체 배열 위로 작동하도록 설계된 내장 판다스 함수를 위해 변환된다. 벡터화는 전체 배열 위로 작업을 실행하는 프로세스다.

판다스는 수학 연산에서 집계 및 문자열 함수(사용 가능한 함수의 광범위한 목록은 판다스 문서에서 확인해라)에 이르기까지 다양한 벡터화 함수를 포함하고 있다. 내장 함수는 판다스 시리즈와 데이터 프레임에서 작동하게끔 최적화되어있다. 결과적으로 벡터화 판다스 함수를 사용하는 건 비슷한 목적을 위해 손수 반복시키는 방법보다 거의 항상 바람직하다.

지금까지는 Haversine 함수에 스칼라를 전달했다. 그러나 Haversine 함수 내에서 사용하는 모든 함수를 배열 위로 작동시킬 수 있다. 이렇게 하면 거리 함수를 매우 간단하게 벡터화할 수 있다. 스칼라 값으로 각 위도, 경도를 전달하는 대신 전체 시리즈(열)를 전달한다. 이를 통해 판다스는 벡터화 함수에 적용 가능한 모든 최적화 옵션을 활용할 수 있고 특히 전체 배열에 대한 모든 계산을 동시에 수행하게 된다.

In [12]:
# Vectorized implementation of Haversine applied on Pandas series
%timeit df['distance'] = haversine(40.671, -73.985,\
                                   df['latitude'], df['longitude'])

525 µs ± 3.92 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


함수 벡터화를 통해 apply() 메서드 대비 50배 이상 개선시켰고 iterrows() 대비 100배 이상 개선시켰다. 입력 유형 변경하는 것 외에 아무것도 하지 않아도 됐다!

## Vectorized implementation of Haversine applied on NumPy arrays

이 지점에서 그만두어도 괜찮다. 판다스 시리즈를 사용해 벡터화하면 상시 계산을 위한 최적화 요구 사항의 거의 대부분을 만족시킬 수 있다. 그러나 속도가 최우선이라면 넘파이 파이썬 라이브러리 형식에 도움을 요청해볼 수 있다.

넘파이 라이브러리는 “과학 계산을 위한 파이썬 기본 패키지”를 표방하며 내부가 최적화된, 사전 컴파일된 C 코드로 작업을 수행한다. 판다스와 마찬가지로 넘파이는 배열 객체(ndarrays라고 함) 상에서 작동한다. 그러나 색인, 데이터 유형 확인 등과 같이 판다스 시리즈 작업으로 인한 오버헤드가 많이 발생하지 않는다. 결과적으로 넘파이 배열에 대한 작업은 판다스 시리즈에 대한 작업보다 훨씬 빠르다.

판다스 시리즈가 제공하는 추가 기능이 중요하지 않을 때 넘파이 배열을 판다스 시리즈 대신 사용할 수 있다. 예를 들어 Haversine 함수의 벡터화 구현은 실제로 위도 또는 경도 시리즈의 색인을 사용하지 않으므로 사용할 수 있는 색인이 없어도 함수가 중단되지 않는다. 이에 비해 색인으로 값을 참조해야 하는 데이터 프레임의 조인 같은 작업을 수행한다면 판다스 개체를 계속 사용하는 편이 낫다.

위도와 경도 배열을 시리즈의 values 메서드를 단순 사용해서 판다스 시리즈에서 넘파이 배열로 변환한다. 시리즈의 벡터화와 마찬가지로 넘파이 배열을 함수에 직접 전달하면 판다스가 전체 벡터에 함수를 적용시킨다.

#### Timing vectorized implementation

In [13]:
# Vectorized implementation of Haversine applied on NumPy arrays
%timeit df['distance'] = haversine(40.671, -73.985,\
                         df['latitude'].values, df['longitude'].values)

60.2 µs ± 507 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [14]:
%%timeit
# Convert pandas arrays to NumPy ndarrays
np_lat = df['latitude'].values
np_lon = df['longitude'].values

2.28 µs ± 32 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Summary

판다스 코드 최적화에 관해 몇 가지 기본적인 결론을 내릴 수 있다.

1 . 반복을 피해라. 사용 사례 대부분의 경우 반복은 느리고 불필요하다.  
2 . 반복해야 하는 경우 반복 함수가 아닌 apply()를 사용해라.  
3 . 보통은 벡터화가 스칼라 연산보다 낫다. 대부분의 판다스 작업은 벡터화시킬 수 있다.  
4 . 넘파이 배열에서의 벡터 연산은 판다스 시리즈에서 수행하는 것보다 효율적이다.  

현재 나는 40.671,-73.985 
직선거리 기준으로 200보다 작고
ㄴ

In [16]:
print(haversine(40.671, -73.985))

TypeError: haversine() missing 2 required positional arguments: 'lat2' and 'lon2'

In [31]:
distance_list = haversine_looping(df)
distance_list #직선거리 LIST


[139.60718990468314,
 139.7468975690499,
 142.19105019740348,
 137.27554601978198,
 139.6845833975413,
 142.51142702074702,
 139.22365519164777,
 137.50441406907777,
 141.9774911345873,
 138.91955469651396,
 141.11563069456204,
 137.3203860002506,
 136.97856807946653,
 139.8973388410462,
 142.05019614681575,
 141.0701358852327,
 141.6853330520409,
 141.73337335126004,
 139.0531985378796,
 139.5583855858655,
 141.28704958482214,
 142.17320162032993,
 142.02076267423007,
 142.21807025310167,
 138.92504179217497,
 142.266292707799,
 139.63740538641187,
 140.72789587052682,
 141.77541531435506,
 141.75768752417588,
 136.94461742445353,
 139.55973230922967,
 141.64840710506382,
 139.18874222243042,
 141.79943053181353,
 141.24794000213632,
 137.23991025543913,
 137.30738891035375,
 141.57618393664677,
 253.25271853949954,
 222.047039471067,
 294.678826493047,
 294.78233777328677,
 294.7895804966099,
 294.77554001935584,
 294.4669163600977,
 294.70595528804364,
 296.45886666124244,
 295.3508

조건문 이내에서 호텔을 출력하는

In [24]:
filtered_df = df[(df['star_rating'] >= 4) & (pd.Series(distance_list) <= 200)]
filtered_df

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate,distance
114,482259,Topping Rose House,One Bridgehampton - Sag Harbor Turnpike,Bridgehampton,NY,11932,40.93780,-72.30069,4.5,995.3900,295.3900,90.001264
129,339951,Sheraton Brooklyn New York Hotel,228 Duffield Street,Brooklyn,NY,11201,40.69160,-73.98436,4.0,409.1485,259.0916,1.423805
134,406834,McCarren Hotel & Pool,160 N 12th St,Brooklyn,NY,11249,40.72100,-73.95543,4.0,575.4100,330.4100,3.786288
142,403899,The Box House Hotel,77 Box Street,Brooklyn,NY,11222,40.73749,-73.95329,4.0,379.4200,249.4200,4.885345
154,136344,New York Marriott at the Brooklyn Bridge,333 Adams St,Brooklyn,NY,11201,40.69365,-73.98870,4.0,489.0700,212.0400,1.577023
...,...,...,...,...,...,...,...,...,...,...,...,...
1592,333528,"Viana Hotel & Spa, BW Premier Collection",3998 Brush Hollow Rd,Westbury,NY,11590,40.77629,-73.55967,4.0,509.9900,133.9500,23.431169
1600,324657,"The Ritz-Carlton New York, Westchester",3 Renaissance Square,White Plains,NY,10601,41.03257,-73.76664,5.0,479.4042,349.2972,27.466891
1601,514006,Furnished Quarters Bank Street Commons,15/25 Bank Street,White Plains,NY,10606,41.03050,-73.77419,4.0,180.0000,179.0000,27.174550
1602,566642,Global Luxury Apartments in White Plains,15 Bank Street,White Plains,NY,10606,41.03150,-73.77401,4.0,175.2200,125.1500,27.241501
